In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

数据的读取，是通过Dataloader
* sampler-->index,
* DataSet-->img,label

In [2]:
from torch.utils.data import DataLoader

DataLoader(dataset,batch_size=1,shuffle=False,sampler=None,batch_sampler=None,pin_memory=False,drop_last=False,timeout=0,worker_init_fn=None)

功能：构建可以迭代的数据装载器
* datasaet:Dataset类，决定数据从哪里读取，如何读取
* batch——size:批大小
* num_works:是否多进程读取数据
* shuffle:每个epoch是否乱序
* drop_last:当样本数不能被batch——size整除时，是否舍弃最后一批数据

In [3]:
from torch.utils.data import Dataset

功能：Dataset抽象类，所有自定义的Dataset需要继承它，并且复写  
__getitem__()  
getitem:  
接收一个inde，返回一个样本

In [4]:
import os

对数据进行处理，制作annotation

In [5]:
def makedir(new_dir):
    if not os.path.exists(new_dir):
        os.makedirs(new_dir)

In [6]:
data_dir = '/home/zhanggong-study/pytorch_tutorial/RMB_data'
train_pct = 0.8
test_pct = 0.2

In [7]:
import glob
import cv2

In [8]:
for name in ['1','100']:
    image_dir=os.path.join(data_dir,name)
    image_list = glob.glob(image_dir+'/*')
    for i,image_name in enumerate(image_list):
        image = cv2.imread(image_name)
        if name=='1':
            img_info = image_name +' '+'0'+'\n'
        else:
            img_info = image_name +' '+'1'+'\n'
        if image.shape==None:
            break
        if i<len(image_list)*0.8:
            with open('/home/zhanggong-study/pytorch_tutorial/RMB_data/train.txt','a') as f:
                f.write(img_info)
        else:
            with open('/home/zhanggong-study/pytorch_tutorial/RMB_data/test.txt','a') as f:
                f.write(img_info)

自定义数据集(此章重点）

In [26]:
class Mydata(Dataset):
    def __init__(self,data_dir,annotation_dir,transform=None):
        self.data_dir = data_dir
        self.annotation_dir =annotation_dir
        self.transform= transform
        with open(self.annotation_dir) as f:
            self.img_info = f.readlines()
    def __len__(self):
        return len(self.img_info)
    def __getitem__(self,index):
        sample ={}
        image_path = self.img_info[index].split()[0]
        img_label = int(self.img_info[index].split()[-1])
        image_BGR =cv2.imread(image_path)
        image_RGB = cv2.cvtColor(image_BGR,cv2.COLOR_BGR2RGB)
        sample['image'] = image_RGB
        sample['label'] = img_label
        if self.transform:
            sample['image'] = self.transform(image_RGB)
        return sample